In [ ]:
"""
ZUSAMMENFASSUNG DER QUELLENANGABEN UND KI-VERWENDUNG:

HAUPTQUELLEN:
1. Devlin et al. (2018) "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding"
2. Hugging Face Transformers Library: https://huggingface.co/docs/transformers/
3. PyTorch Documentation: https://pytorch.org/docs/
4. Scikit-learn Documentation: https://scikit-learn.org/
5. Liu et al. (2019) "Multi-Task Deep Neural Networks for Natural Language Understanding"
6. Caruana (1997) "Multitask Learning"
7. Kohavi (1995) "A study of cross-validation and bootstrap for accuracy estimation and model selection"

MODELL-SPEZIFISCHE QUELLEN:
- German BERT Model: https://huggingface.co/bert-base-german-cased
- Uncertainty Quantification: Gal & Ghahramani (2016) "Dropout as a Bayesian Approximation"
- Ensemble Methods: Breiman (1996) "Bagging predictors"

HINWEIS: Teile dieses Codes wurden unter Verwendung von KI-Tools entwickelt,
insbesondere für Optimierung, Strukturierung und Implementierung.
Alle wissenschaftlichen Konzepte basieren auf den oben genannten Publikationen.
"""

####### IMPORTIERTE PAKETE UND MODULE ########
# Laden der notwendigen Python Bibliotheken, um mit Datensatz zu arbeiten
import os
from copy import deepcopy

# Die Pandas-Bibliothek wird hier als "pd" importiert.
# Pandas ist eine leistungsstarke Bibliothek zur Datenverarbeitung und -analyse. Sie erleichtert
# das Laden, Bearbeiten und Speichern von tabellenartigen Daten, z. B. aus CSV-Dateien.
import pandas as pd

# Die Transformers-Bibliothek bietet eine Sammlung vortrainierter Modelle für verschiedene NLP-Aufgaben,
# wie Textklassifikation, Textgenerierung und Übersetzung. Hier wird speziell die "pipeline"-Funktion importiert,
# die eine vereinfachte Möglichkeit bietet, vortrainierte Modelle zu verwenden.
from transformers import pipeline

# NLTK (Natural Language Toolkit) ist eine Bibliothek zur Sprachverarbeitung, die eine Vielzahl an
# Tools und Datensätzen für natürliche Sprachverarbeitung (Natural Language Processing, NLP) enthält,
# wie z. B. Tokenizer, Wortlisten und Korpora.
import nltk

# Die Module `DataLoader` und `Dataset` aus `torch.utils.data` dienen zur Handhabung von Datensätzen.
# "Dataset" bietet eine Standardmethode zur Definition eines Datensatzes, während "DataLoader" Funktionen
# zum effizienten Laden, Shuffeln und Batchen der Daten bereitstellt. Diese Module sind zentral für das Training von Modellen.
from torch.utils.data import DataLoader, Dataset

# Die PyTorch-Bibliothek wird hier importiert, die eine der populärsten Bibliotheken für Deep Learning ist.
# Sie bietet grundlegende Werkzeuge für die Arbeit mit neuronalen Netzen, Tensoren und GPU-unterstütztem Rechnen.
import torch

# Hier werden Module für neuronale Netzwerke und die Optimierung in PyTorch importiert.
# `torch.nn` enthält Bausteine für den Aufbau neuronaler Netze,
# während `torch.optim` Werkzeuge für die Optimierung von Gewichten im Trainingsprozess bereitstellt.
import torch.nn as nn
import torch.optim as optim

# `LabelEncoder` wird aus sklearn.preprocessing importiert.
# Dieser Encoder wird verwendet, um kategorische Labels (z.B. "positive", "negative") in numerische Werte umzuwandeln,
# da maschinelle Lernmodelle nur numerische Daten verarbeiten können.
from sklearn.preprocessing import LabelEncoder

# `train_test_split` aus `sklearn.model_selection` trennt die Daten in Trainings- und Testdatensätze.
# Dies ist eine Standardtechnik, um die Modellleistung zu testen und Überanpassung (Overfitting) zu vermeiden.
from sklearn.model_selection import train_test_split, KFold

# Die Funktion `pad` aus `torch.nn.functional` wird importiert. Sie dient dazu, Eingaben mit unterschiedlicher Länge
# auf eine einheitliche Länge zu bringen, indem sie am Ende der Sequenzen Padding-Zeichen einfügt.
# Dies ist besonders nützlich für Textdaten, da diese in der Regel variierende Längen aufweisen.
from torch.nn.functional import pad

# `BertForSequenceClassification` ist ein vortrainiertes BERT-Modell aus der Transformers-Bibliothek,
# das speziell für Klassifikationsaufgaben angepasst ist. Es kann verwendet werden, um Textsequenzen
# in Kategorien einzuordnen, nachdem es auf spezifische Daten trainiert wurde.
from transformers import BertForSequenceClassification

# Der `BertTokenizer` wird verwendet, um Eingabetexte in Token umzuwandeln, die das Modell verstehen kann.
# Tokenizer teilen Text in einzelne Bestandteile (Tokens) auf und konvertieren sie in IDs,
# die als Eingabe für das BERT-Modell dienen.
from transformers import BertTokenizer

# Import des BERT-Modells und des Tokenizers aus der Transformers-Bibliothek
# `BertModel` ist das BERT-Sprachmodell, das kontextuelle Darstellungen von Text erstellt.
# `BertTokenizer` konvertiert Texte in Token und Token-IDs, die BERT als Eingabe verarbeitet.
from transformers import BertModel, BertTokenizer
import numpy as np

# Installation der benötigten Bibliotheken
!pip install transformers

In [ ]:
####### HILFSFUNKTIONEN ########
# Mapping der Themenkategorien zu Mitarbeitern


mitarbeiter_map = {
    'Lohn': 'Alexandra Himmelreich',
    'EDV': 'Luke Horchler',
    'FiBu': 'Sven Althaus',
    'Unternehmensberatung': 'Stephan Sonneborn',
    'Einkommensteuer': 'Yvonne Isken'
}

# Aufteilung nach Themen
def split_by_topic(df, test_samples_per_topic=1, random_state=42):
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for topic in df['Thema'].unique():
        topic_df = df[df['Thema'] == topic]
        topic_test_df = topic_df.sample(n=test_samples_per_topic, random_state=random_state)
        topic_train_df = topic_df.drop(topic_test_df.index)
        train_df = pd.concat([train_df, topic_train_df])
        test_df = pd.concat([test_df, topic_test_df])
    return train_df, test_df

# Aufteilung in gleichmäßige Subsets für Cross-Validation
def split_train_into_subsets(df, num_subsets=10, random_state=42):
    subsets = [pd.DataFrame() for _ in range(num_subsets)]
    for topic in df['Thema'].unique():
        topic_df = df[df['Thema'] == topic].sample(frac=1, random_state=random_state)
        topic_splits = np.array_split(topic_df, num_subsets)
        for i in range(num_subsets):
            subsets[i] = pd.concat([subsets[i], topic_splits[i]])
    return subsets

# Funktion zur Vorhersage und Behandlung von Unsicherheiten
# QUELLE: Inspiriert von Uncertainty Quantification in Deep Learning (Gal & Ghahramani, 2016)
def predict_and_handle_uncertainty(model, question, true_answer="", true_category="", threshold=0.4):
    mitarbeiter_map = {
      'Lohn': 'Alexandra Himmelreich',
      'EDV': 'Luke Horchler',
      'FiBu': 'Sven Althaus',
      'Unternehmensberatung': 'Stephan Sonneborn',
      'Einkommensteuer': 'Yvonne Isken'
    }
    model.eval()
    encoding = tokenizer.encode_plus(
        question,
        truncation=True,
        max_length=200,
        padding='max_length',
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        # Vorhersage durchführen
        answer_logits, category_logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # Softmax-Wahrscheinlichkeiten berechnen
        answer_probs = torch.softmax(answer_logits, dim=1)
        category_probs = torch.softmax(category_logits, dim=1)

        # Maximale Wahrscheinlichkeit und zugehörigen Index bestimmen
        answer_confidence, answer_pred = torch.max(answer_probs, dim=1)
        category_confidence, category_pred = torch.max(category_probs, dim=1)

    # Vorhersagen in lesbare Ausgaben umwandeln
    if answer_confidence.item() < threshold:
        answer_pred_label = "Unable to provide a confident answer:", answers_encoder.inverse_transform(answer_pred.cpu().numpy())[0]
    else:
        answer_pred_label = answers_encoder.inverse_transform(answer_pred.cpu().numpy())[0]

    if category_confidence.item() < threshold:
        category_pred_label = "Unable to determine category"
    else:
        category_pred_label = category_encoder.inverse_transform(category_pred.cpu().numpy())[0]

    # Ausgabe der Vorhersageergebnisse
    print(f"Predicted Answer: {answer_pred_label}")
    print(f"Predicted Category: {category_pred_label}")

    if true_answer and true_category:
        print(f"Correct Answer: {true_answer}")
        print(f"Correct Category: {true_category}")

    print(f"Answer Prediction Confidence: {answer_confidence.item()}")
    print(f"Category Prediction Confidence: {category_confidence.item()}")

    # Entscheidung über die Ausgabe basierend auf Konfidenzwerten
    message = ""
    if answer_confidence.item() < threshold and category_confidence.item() < threshold:
        message = "Leider kann ich das nicht bestimmen. Bitte wenden Sie sich an unser Support-Team für Unterstützung."
    elif answer_confidence.item() < threshold and category_confidence.item() >= threshold:
        category_pred_label = category_pred_label.strip()
        if category_pred_label in mitarbeiter_map:
            message = f"Bitte wenden Sie sich an {mitarbeiter_map[category_pred_label]}!"
        else:
            message = "Leider kann ich das nicht bestimmen. Bitte wenden Sie sich an unser Support-Team für Unterstützung."
    else:
        message = answer_pred_label

    print(message + "\n")
    return message

# Trainingsfunktion für Multi-Task Learning
# QUELLE: Multi-Task Learning Konzepte basierend auf Caruana (1997) "Multitask Learning"
def train_multi_task_model(model, dataloader, criterion_answer, criterion_category, optimizer, num_epochs=10):
    model.train()  # Setzt das Modell in den Trainingsmodus
    training_accurcy=[]

    for epoch in range(num_epochs):
        epoch_loss = 0
        correct_answers = 0
        correct_categories = 0
        total = 0

        for batch in dataloader:
            input_ids, attention_mask, answer_labels, category_labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            answer_labels = answer_labels.to(device)
            category_labels = category_labels.to(device)

            optimizer.zero_grad()

            # Vorwärtsdurchlauf
            answer_outputs, category_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Verlustberechnung mit gewichteter Kombination
            # HINWEIS: Gewichtung 0.5 für Kategorieverlust
            loss_answer = criterion_answer(answer_outputs, answer_labels)
            loss_category = criterion_category(category_outputs, category_labels)
            loss = loss_answer + loss_category *0.5

            # Backward-Pass
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Berechnen der Genauigkeit
            _, predicted_answers = torch.max(answer_outputs, 1)
            _, predicted_categories = torch.max(category_outputs, 1)

            total += answer_labels.size(0)
            correct_answers += (predicted_answers == answer_labels).sum().item()
            correct_categories += (predicted_categories == category_labels).sum().item()

        # Genauigkeit für die Epoche berechnen
        epoch_answer_accuracy = correct_answers / total
        epoch_category_accuracy = correct_categories / total
        training_accurcy.append((epoch_answer_accuracy, epoch_category_accurcy))
        # Ausgabe der Trainingsergebnisse
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}, Answer Accuracy: {epoch_answer_accuracy:.4f}, Category Accuracy: {epoch_category_accuracy:.4f}")

    return model, training_accurcy

# Evaluationsfunktion
# QUELLE: Standard Model Evaluation Patterns aus PyTorch Documentation
def evaluate_multi_task_model(model, dataloader, criterion_answer, criterion_category):
    model.eval()  # Setzt das Modell in den Evaluationsmodus
    eval_loss = 0
    correct_answers = 0
    correct_categories = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, answer_labels, category_labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            answer_labels = answer_labels.to(device)
            category_labels = category_labels.to(device)

            # Vorwärtsdurchlauf
            answer_outputs, category_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Verlustberechnung
            loss_answer = criterion_answer(answer_outputs, answer_labels)
            loss_category = criterion_category(category_outputs, category_labels)
            loss = loss_answer + loss_category
            eval_loss += loss.item()

            # Berechnen der Genauigkeit
            _, predicted_answers = torch.max(answer_outputs, 1)
            _, predicted_categories = torch.max(category_outputs, 1)

            total += answer_labels.size(0)
            correct_answers += (predicted_answers == answer_labels).sum().item()
            correct_categories += (predicted_categories == category_labels).sum().item()

    # Ausgabe der Evaluationsergebnisse
    avg_loss = eval_loss / len(dataloader)
    answer_accuracy = correct_answers / total
    category_accuracy = correct_categories / total
    print(f"Validation Loss: {avg_loss:.4f}, Answer Accuracy: {answer_accuracy:.4f}, Category Accuracy: {category_accuracy:.4f}")

    return answer_accuracy, category_accuracy

In [ ]:
####### DEFINITION DES KLASSIFIKATIONSMODELLS ########
# Festlegen der GPU oder CPU für das Training
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')
# Definition der MultiTaskBERTClassifier-Klasse
# QUELLE: Multi-Task Learning Architecture inspiriert von Liu et al. (2019) "Multi-Task Deep Neural Networks for Natural Language Understanding"
class MultiTaskBERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size=768, num_answer_labels=10, num_category_labels=5):
        super(MultiTaskBERTClassifier, self).__init__()

        # Übernimmt das übergebene BERT-Modell als Basismodell für die Textrepräsentation
        # QUELLE: BERT-Basis von Hugging Face Transformers
        self.bert = bert_model

        # Answer prediction head
        # Ein Klassifikator für die Vorhersage von Antwortkategorien
        self.answer_classifier = nn.Sequential(
            # von 128 zu 256
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            # von 128 zu 256
            nn.Linear(256, num_answer_labels)
        )

        # Category prediction head
        # Ein Klassifikator für die Vorhersage der Themenkategorie
        self.category_classifier = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, num_category_labels)
        )

    def forward(self, input_ids, attention_mask):
        # Extrahieren der CLS-Token-Darstellung
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # Repräsentation des CLS-Tokens

        # Weitergabe durch Klassifikator-Köpfe
        answer_logits = self.answer_classifier(cls_token)
        category_logits = self.category_classifier(cls_token)

        return answer_logits, category_logits

# Definition der QADataset-Klasse
# QUELLE: PyTorch Dataset Pattern aus offizieller Dokumentation
class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_len=200):
        self.tokenizer = tokenizer
        self.max_len = max_len

        self.questions = data['Frage'].tolist()
        self.response_labels = data['Antwort'].tolist()   # Already encoded as integers
        self.category_labels = data['Thema'].tolist()     # Already encoded as integers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        response_label = self.response_labels[idx]
        category_label = self.category_labels[idx]

        encoding = self.tokenizer.encode_plus(
            question,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return input_ids, attention_mask, torch.tensor(response_label, dtype=torch.long), torch.tensor(category_label, dtype=torch.long)

In [ ]:
####### IMPORTIEREN DER DATEN ########
# Download von NLTK-Ressourcen

nltk.download('wordnet')
nltk.download('omw-1.4')

# Festlegen des aktuellen Verzeichnisses
current_directory = os.getcwd()

file_name = "FAQs - Liste.xlsx"

# Datei in ein Pandas DataFrame laden
# Die benötigten Spalten festlegen
required_columns = ['Frage', 'Antwort', 'Thema']

df = pd.read_excel(file_name, engine='openpyxl', usecols=required_columns)
df = df.dropna(subset=required_columns)
print(df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


             Thema                                              Frage  \
1  Einkommensteuer        Muss ich den Original-Bescheid aufbewahren?   
2  Einkommensteuer            Muss ich den Originalbescheid behalten?   
3  Einkommensteuer  Ist es erforderlich, den Originalbescheid aufz...   
4  Einkommensteuer   Soll ich den Originalbescheid lange aufbewahren?   
5  Einkommensteuer  Muss ich den Originalbescheid für immer aufbew...   

    Antwort  
1  §257 HGB  
2  §257 HGB  
3  §257 HGB  
4  §257 HGB  
5  §257 HGB  


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
####### INITIALISIERUNG DES MODELLS UND TOKENIZERS ########

# Initialisierung des BERT-Modells und des Tokenizers
# QUELLE: Deutsche BERT-Modelle von https://huggingface.co/bert-base-german-cased
# HINWEIS: German BERT für deutsche Texte optimiert
model_name = 'bert-base-german-cased'
bert_model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# DataFrame vor der Verwendung von train_df in Trainings- und Testdatensätze aufteilen
from sklearn.model_selection import train_test_split

# Thema ist die Spalte, nach der geschichtet werden soll
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Thema'])

# Nutzung von train_df
num_answer_labels = len(train_df['Antwort'].unique())
num_category_labels = len(train_df['Thema'].unique())

questions_encoder = LabelEncoder()
answers_encoder = LabelEncoder()
answers_encoder.fit(train_df['Antwort'])
num_answer_labels = len(answers_encoder.classes_)

category_encoder = LabelEncoder()
category_encoder.fit(train_df['Thema'])
num_category_labels = len(category_encoder.classes_)
bert_model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Bestimmen der Anzahl von Antwort- und Kategorielabels
num_answer_labels = len(train_df['Antwort'].unique())
num_category_labels = len(train_df['Thema'].unique())

questions_encoder = LabelEncoder()
answers_encoder = LabelEncoder()
answers_encoder.fit(df['Antwort'])
num_answer_labels = len(answers_encoder.classes_)

category_encoder = LabelEncoder()
category_encoder.fit(df['Thema'])
num_category_labels = len(category_encoder.classes_)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/485k [00:00<?, ?B/s]

In [ ]:
####### FUNKTION FÜR K-FOLD-KREUZVALIDIERUNG ########
# QUELLE: K-Fold Cross-Validation nach Kohavi (1995) "A study of cross-validation and bootstrap for accuracy estimation and model selection"

kf = KFold(n_splits = 10, shuffle=True, random_state=42)

questions = df['Frage'].tolist()
answers = df['Antwort'].tolist()
categories = df['Thema'].tolist()

answers_labels = answers_encoder.transform(answers)
category_labels = category_encoder.transform(categories)

data_tuples = list(zip(questions, answers_labels, category_labels))
data_tuples = np.array(data_tuples)

# Definition der Verlustfunktionen
criterion_answer = nn.CrossEntropyLoss()
criterion_category = nn.CrossEntropyLoss()
train_accuracies_per_fold = []


fold = 1
for train_index, val_index in kf.split(data_tuples):
  train_data = data_tuples[train_index]
  val_data = data_tuples[val_index]

  train_df_fold = pd.DataFrame(train_data, columns = ['Frage', 'Antwort', 'Thema'])
  val_df_fold = pd.DataFrame(val_data, columns = ['Frage', 'Antwort', 'Thema'])

  train_df_fold['Antwort'] = train_df_fold['Antwort'].astype(int)
  train_df_fold['Thema'] = train_df_fold['Thema'].astype(int)
  val_df_fold['Antwort'] = val_df_fold['Antwort'].astype(int)
  val_df_fold['Thema'] = val_df_fold['Thema'].astype(int)

  train_dataset = QADataset(train_df_fold, tokenizer)
  train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

  val_dataset = QADataset(val_df_fold, tokenizer)
  val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

  model = MultiTaskBERTClassifier(
    deepcopy(bert_model),
    hidden_size=768,
    num_answer_labels=num_answer_labels,
    num_category_labels=num_category_labels
  ).to(device)

  # Hyperparameter Anpassung
  # HINWEIS: Learning Rate empirisch von 2e-5 auf 1e-5 reduziert für stabileres Training
  optimizer = optim.Adam(model.parameters(), lr=1e-5) # vorher 2le-5
  trained_model, train_accuracies = train_multi_task_model(
            model=model,
            dataloader=train_loader,
            criterion_answer=criterion_answer,
            criterion_category=criterion_category,
            optimizer=optimizer,
            num_epochs=10
        )
  train_accuracies_per_fold.append(train_accuracies)

  # Erstellt einen Tracker für Validierungsmetriken

  val_metrics = {
      'answer_correct': 0,
      'category_correct': 0,
      'total': 0
  }

  # Modell in den Evaluierungsmodus versetzen.
  trained_model.eval()

  with torch.no_grad():
    for batch_idx, (input_ids, attention_mask, answers_label, category_label) in enumerate(val_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        answers_label = answers_label.to(device)
        category_label = category_label.to(device)

        # Modellvorhersagen erhalten
        answer_logits, category_logits = trained_model(input_ids, attention_mask)

        # Abrufen der vorhergesagten Klassen
        answer_preds = torch.argmax(answer_logits, dim=1)
        category_preds = torch.argmax(category_logits, dim=1)

        # Berechnung der Genauigkeit für batch
        answer_correct = (answer_preds == answers_label).sum().item()
        category_correct = (category_preds == category_label).sum().item()

        # Metriken aktualisieren
        val_metrics['answer_correct'] += answer_correct
        val_metrics['category_correct'] += category_correct
        val_metrics['total'] += input_ids.size(0)

        # Print Beispielprognosen (nur für den ersten Batch)
        if batch_idx == 0:
            for i in range(min(3, input_ids.size(0))):
                question = tokenizer.decode(input_ids[i], skip_special_tokens=True)
                true_answer_idx = answers_label[i].item()
                pred_answer_idx = answer_preds[i].item()
                true_category_idx = category_label[i].item()
                pred_category_idx = category_preds[i].item()

                true_answer = answers_encoder.classes_[true_answer_idx]
                pred_answer = answers_encoder.classes_[pred_answer_idx]
                true_category = category_encoder.classes_[true_category_idx]
                pred_category = category_encoder.classes_[pred_category_idx]

                print(f"\nQuestion: {question}")
                print(f"True Answer: {true_answer}")
                print(f"Predicted Answer: {pred_answer}")
                print(f"True Category: {true_category}")
                print(f"Predicted Category: {pred_category}")


    # Berechnung der Gesamtgenauigkeit der Validierung
    answer_accuracy = val_metrics['answer_correct'] / val_metrics['total']
    category_accuracy = val_metrics['category_correct'] / val_metrics['total']

    print(f"\nFold {fold} Validation Results:")
    print(f"Answer Accuracy: {answer_accuracy:.4f}")
    print(f"Category Accuracy: {category_accuracy:.4f}")

    # Model speichern
    model_path = f"model_fold_{fold}.pt"
    torch.save(trained_model.state_dict(), model_path)
    print(f"Saved model for fold {fold} to {model_path}")


  fold += 1


Epoch [1/10], Loss: 5.2933, Answer Accuracy: 0.0375, Category Accuracy: 0.5385
Epoch [2/10], Loss: 4.5659, Answer Accuracy: 0.2962, Category Accuracy: 0.8327
Epoch [3/10], Loss: 3.7913, Answer Accuracy: 0.6769, Category Accuracy: 0.9615
Epoch [4/10], Loss: 3.0770, Answer Accuracy: 0.8750, Category Accuracy: 0.9837
Epoch [5/10], Loss: 2.4422, Answer Accuracy: 0.9385, Category Accuracy: 0.9923
Epoch [6/10], Loss: 1.9067, Answer Accuracy: 0.9663, Category Accuracy: 0.9913
Epoch [7/10], Loss: 1.4597, Answer Accuracy: 0.9769, Category Accuracy: 0.9952
Epoch [8/10], Loss: 1.1247, Answer Accuracy: 0.9827, Category Accuracy: 0.9981
Epoch [9/10], Loss: 0.8815, Answer Accuracy: 0.9837, Category Accuracy: 0.9981
Epoch [10/10], Loss: 0.6883, Answer Accuracy: 0.9894, Category Accuracy: 0.9981

Question: Was ist zu beachten, um die Nachweisbarkeit einer Rechnung bei einer Ausfuhr von Waren oder Dienstleistungen zu gewährleisten?
True Answer: siehe unser Merkblatt "Buch - und Belegnachweise Innergeme

In [ ]:
# Testfragen für Modell-Evaluation
# HINWEIS: Mix aus domänenspezifischen und Out-of-Domain Fragen für Robustheitstesting
test_fragen = ["Wie gehe ich vor, wenn Unternehmen Online auf meinem Endgerät nicht reagiert?",
               "Welche Browsereinstellungen könnten den Start von Unternehmen Online verhindern?",
               "Was sind typische Fehlerquellen, die den Start von Unternehmen Online blockieren?",
               "Was tun, wenn meine Software nur startet, wenn ich gleichzeitig Jodeln kann?",
               "Warum besteht mein digitales Werkzeug darauf, meine Passwörter in Piratensprache zu übersetzen?",
               "Wie reagiere ich, wenn mein Programm eine Existenzkrise hat und sich für eine Karriere als Videospiel interessiert?",
               "Was kann ich tun wenn ich eine Existenzkrise habe und mich für eine Karriere als Videospiel interessiere?"]

# Ensemble-Testing: Evaluation aller trainierten Modelle
# QUELLE: Ensemble Methods nach Breiman (1996) "Bagging predictors"
for fold in range(1, 11):
    # Neue Model Instanzen erstellen
    model = MultiTaskBERTClassifier(
        deepcopy(bert_model),
        hidden_size=768,
        num_answer_labels=num_answer_labels,
        num_category_labels=num_category_labels
    ).to(device)

    # Model laden
    model_path = f"model_fold_{fold}.pt"
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print (f"***** FOLD {fold} *****")

    # Vorhersagen generieren
    for question in test_fragen:
        print(f"Question: {question}")
        predict_and_handle_uncertainty(model, question, threshold=0.4)


***** FOLD 1 *****
Question: Wie gehe ich vor, wenn Unternehmen Online auf meinem Endgerät nicht reagiert?
Predicted Answer: Löschen Sie im Browser den Verlauf und die temporären Internetdatein über den gesamten Verlauf.  Sollte das keinen Erfolg haben, wenden Sie sich an unser Suppoert-Team.
Predicted Category: EDV 
Answer Prediction Confidence: 0.6091829538345337
Category Prediction Confidence: 0.9610942006111145
Löschen Sie im Browser den Verlauf und die temporären Internetdatein über den gesamten Verlauf.  Sollte das keinen Erfolg haben, wenden Sie sich an unser Suppoert-Team.

Question: Welche Browsereinstellungen könnten den Start von Unternehmen Online verhindern?
Predicted Answer: ('Unable to provide a confident answer:', 'Du kannst in Verbindung mit SmartLogin  nahezu mit jedem Endgerät Unternehmen Online erreichen. Bei der Nutzung von iOS /MacOS /Android musst du auf ein paar Besonderheiten achten, dazu gibt es im Unternehmen Online unter "Hilfe" gute Tips.')
Predicted Catego

In [ ]:
# Visualisierung der Trainingsergebnisse
# QUELLE: Matplotlib für wissenschaftliche Visualisierungen
import matplotlib.pyplot as plt

#answer accuracy
plt.figure(figsize=(12, 5))

for fold_idx, train_accuracies in enumerate(train_accuracies_per_fold):
    answer_acc = [acc[0] for acc in train_accuracies]
    plt.plot(answer_acc, label=f"Fold {fold_idx+1}")

plt.title("Answer Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#category accuracy
plt.figure(figsize=(12, 5))

for fold_idx, train_accuracies in enumerate(train_accuracies_per_fold):
    category_acc = [acc[1] for acc in train_accuracies]
    plt.plot(category_acc, label=f"Fold {fold_idx+1}")

plt.title("Category Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Training des Models auf der kompletten Datenmenge (ohne k-Fold Methode) alle Daten werden betrachtet
# Aufteilen der Daten in Trainings- und Testdaten
train_df, test_df = split_by_topic(df, test_samples_per_topic=2, random_state=42)

# Initialisierung des Multi-Task-Klassifikators
model = MultiTaskBERTClassifier(
    bert_model,
    hidden_size=768,
    num_answer_labels=num_answer_labels,
    num_category_labels=num_category_labels
).to(device)


train_df['Antwort'] = answers_encoder.transform(train_df['Antwort'])
train_df['Thema'] = category_encoder.transform(train_df['Thema'])

test_df['Antwort'] = answers_encoder.transform(test_df['Antwort'])
test_df['Thema'] = category_encoder.transform(test_df['Thema'])

# Erstellen der Datasets und DataLoader für Training und Test
train_dataset = QADataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = QADataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Definition der Verlustfunktionen
criterion_answer = nn.CrossEntropyLoss()
criterion_category = nn.CrossEntropyLoss()

# Initialisierung des Optimierers
# HINWEIS: Learning Rate reduziert für stabiles Training (empirisch bestimmt)
optimizer = optim.Adam(model.parameters(), lr=1e-5) # vorher 2le-5 Hyperparameter Anpassung

# Training des Modells durchführen
print("Starte Training...")
model = train_multi_task_model(model, train_loader, criterion_answer, criterion_category, optimizer, num_epochs=10) # Hier vorher 20


# Evaluation des trainierten Modells
answer_accuracy, category_accuracy = evaluate_multi_task_model(model, test_loader, criterion_answer, criterion_category)

# Speichern der LabelEncoder für spätere Vorhersagen
# Erstelle neue LabelEncoder-Instanzen und trainiere sie mit allen verfügbaren Daten
label_encoder_answer = LabelEncoder()
label_encoder_answer.fit(df['Antwort'].unique())

label_encoder_category = LabelEncoder()
label_encoder_category.fit(df['Thema'].unique())

# Speichern der LabelEncoder
import pickle
with open('label_encoder_answer.pkl', 'wb') as f:
    pickle.dump(label_encoder_answer, f)
with open('label_encoder_category.pkl', 'wb') as f:
    pickle.dump(label_encoder_category, f)

print("LabelEncoder gespeichert.")


KeyboardInterrupt: 

In [ ]:
# Beispielhafte Vorhersagen
# Hier wird getest, ob das Model den Kontext verstanden hat, obwohl die Frage nicht im Datensatz vorhanden ist.
# HINWEIS: Generalisierungstest mit domänenspezifischen Beispielen
question = "Was muss ins betracht gezogen werden als Werkstudent?"
predict_and_handle_uncertainty(model, question,"Werkstudenten, die neben ihrem Studium arbeiten, sind in der Regel versicherungsfrei in der Kranken-, Pflege- und Arbeitslosenversicherung, wenn sie in der vorlesungsfreien Zeit nicht mehr als 20 Stunden pro Woche arbeiten. Sie sind jedoch in der Rentenversicherung pflichtversichert. Lohnsteuerrechtlich sollte die Besteuerung nach der Lohnsteuerkarte des Studenten erfolgen, und der Student muss sich selbst um eine studentische Krankenversicherung kümmern.", "Lohn")

question = "Was ist ein Unternehmens-Check?"
predict_and_handle_uncertainty(model, question, "Der L&P Unternehmens-Check ist der Einstieg in ein präzises Analyse- und Steuerungssystem für deinen Betrieb. Verbinde eine Standortbestimmung durch die Spezialisten von Lückel & Partner mit der Einführung digitaler Prozesse in deinem Unternehmen. Spezialisten liefern verständliche und aussagekräftige Zahlen und begleiten dich für einen nachhaltigen Erfolg in deinem Unternehmen. https://www.lup-beratung.de/unternehmens-check-holzbau", "Unternehmensberatung")

question = "Darf der Urlaub im Bauhauptgewerbe bei Austritt an den Arbeitnehmer ausgezahlt werden?"
predict_and_handle_uncertainty(model, question, "Nein, der Urlaub wird mitgenommen zum neuen Arbeitgeber. Sollte der Mitarbeiter nicht mehr im Baugewerbe tätig werden, kann er nach 3 Monaten die Urlaubsabgeltung bei der Soka-Bau beantragen.", "Lohn")

print("Fertig.")
